In [7]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)

In [8]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')
df=df.dropna(axis=0)

In [9]:
y=df['EColi.']
X=df.loc[:, ['陰影起伏', '曲率', '傾斜角', '傾斜方向','高度', '近くの川への距離', '鉄道ポイントへの距離', '主要道路への距離', 
             'near_distance_syorizyou', 'supply_hours', 'no water days', 'total population ', 'population served', 'popu-served', 
             'number taps', 'pipes(km)', 'pipes/tabs \n個/km', 'served/pipes', '(popu-served)/pipes\n  /km', 
             'Age of oldeset\npipe ', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [11]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [12]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [13]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

# SVC

In [8]:
from sklearn.svm import SVC
svm=SVC(kernel='linear', C=100, random_state=1)

In [9]:
svm.fit(X_train_std, y_train)

SVC(C=100, kernel='linear', random_state=1)

In [10]:
svm.score(X_train_std, y_train)

0.7171052631578947

In [13]:
svm.score(X_test_std, y_test)

0.6666666666666666

In [1]:
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
pipe_svc=make_pipeline( SVC(random_state=1))

param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid=[{'svc__C':param_range, 'svc__kernel':['linear']},
           ]

In [15]:
gs=GridSearchCV(estimator=pipe_svc,
                param_grid=param_grid,
                scoring='accuracy',
                cv=10,
                n_jobs=-1)

In [16]:
gs=gs.fit(X_train_std, y_train)

In [17]:
gs.best_score_

0.6195833333333333

In [18]:
gs.best_params_

{'svc__C': 100.0, 'svc__kernel': 'linear'}

# ロジスティック回帰

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'C' : [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=10)
grid_search.fit(X_train_std, y_train)

print("Best parameters : {}".format(grid_search.best_params_))
print("Best cross-validation score : {:.3f}".format(gbrid_search.best_score_))

C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

Best parameters : {'C': 1000.0}


C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

NameError: name 'gbrid_search' is not defined

In [21]:
lr=LogisticRegression(C=10, random_state=1)

In [22]:
lr.fit(X_train_std, y_train)

C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10, random_state=1)

In [23]:
lr.score(X_train_std, y_train)

0.7171052631578947

In [24]:
lr.score(X_test_std, y_test)

0.6515151515151515

# random forest

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
forest=RandomForestClassifier(max_depth=15, criterion='entropy', n_estimators=17, random_state=1, max_features=5)

In [73]:
forest=RandomForestClassifier(max_depth=15, criterion='entropy', n_estimators=17, random_state=1)

In [21]:
forest.fit(X_train_std, y_train)

RandomForestClassifier(criterion='entropy', max_depth=15, max_features=5,
                       n_estimators=17, random_state=1)

In [22]:
forest.score(X_train_std, y_train)

0.993421052631579

In [23]:
forest.score(X_test_std, y_test)

0.7727272727272727

In [78]:
from sklearn.ensemble import RandomForestClassifier
feat_labels=X_train_std.columns[0:]
forest=RandomForestClassifier(max_depth=15, criterion='entropy', n_estimators=17, random_state=1)
forest.fit(X_train_std, y_train)
importances=forest.feature_importances_
indices=np.argsort(importances)[::-1]
for f in range(X_train_std.shape[1]):
    print("%2d) %-*s %f" % (f+1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) 鉄道ポイントへの距離                     0.112076
 2) near_distance_syorizyou        0.110401
 3) 高度                             0.108498
 4) 近くの川への距離                       0.091177
 5) 傾斜方向                           0.090476
 6) 傾斜角                            0.083345
 7) 主要道路への距離                       0.077334
 8) 曲率                             0.076102
 9) no water days                  0.046657
10) population served              0.032224
11) pipes/tabs 
個/km               0.027615
12) Age of oldeset
pipe            0.023542
13) 陰影起伏                           0.023448
14) popu-served                    0.019034
15) pipes(km)                      0.016764
16) (popu-served)/pipes
  /km      0.014318
17) number taps                    0.013627
18) served/pipes                   0.011418
19) total population               0.009457
20) supply_hours                   0.007250
21) PF                             0.002386
22) RF                             0.002056
23) FL                          

In [72]:
from sklearn.ensemble import RandomForestClassifier
feat_labels=X_train_std.columns[0:]
forest=RandomForestClassifier(n_estimators=500, random_state=1)
forest.fit(X_train_std, y_train)
importances=forest.feature_importances_
indices=np.argsort(importances)[::-1]
for f in range(X_train_std.shape[1]):
    print("%2d) %-*s %f" % (f+1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) 高度                             0.116199
 2) near_distance_syorizyou        0.104782
 3) 鉄道ポイントへの距離                     0.093165
 4) 近くの川への距離                       0.088888
 5) 主要道路への距離                       0.083489
 6) 傾斜方向                           0.083292
 7) 傾斜角                            0.078709
 8) 曲率                             0.069596
 9) no water days                  0.055235
10) 陰影起伏                           0.033277
11) popu-served                    0.023322
12) pipes(km)                      0.022393
13) population served              0.019870
14) total population               0.017114
15) (popu-served)/pipes
  /km      0.016915
16) number taps                    0.016549
17) pipes/tabs 
個/km               0.016354
18) served/pipes                   0.016353
19) Age of oldeset
pipe            0.016330
20) supply_hours                   0.013818
21) PF                             0.006345
22) RSF                            0.002666
23) ST                          

In [1]:
search_params = {
     'n_estimators'      : [5, 10, 20, 30, 50, 100, 300],
      'max_features'      : [3, 5, 10, 15, 20],
      'random_state'      : [1],
      'min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
      'max_depth'         : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100]
}

In [17]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(RandomForestClassifier(),           # 対象の機械学習モデル
                  search_params,   # 探索パラメタ辞書
                  cv=10,            # クロスバリデーションの分割数
                  verbose=True,    # ログ表示
                  n_jobs=-1)       # 並列処理
gs.fit(X_train_std, y_train)
 
print(gs.best_estimator_)

Fitting 10 folds for each of 3500 candidates, totalling 35000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   18.9s


KeyboardInterrupt: 

In [ ]:
RFC_grid.values()

In [51]:
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
                                       "criterion": ["gini", "entropy"],
                                       "max_depth":[i for i in range(1, 21)],
                                      }

In [52]:
max_depth[0]

NameError: name 'max_depth' is not defined

In [38]:
range(len(RFC_grid["criterion"]))

range(0, 2)

In [40]:
RFC_grid["criterion"][0]

'gini'

In [ ]:
dic

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

max_score = 0
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
#ランダムフォレストの実行
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    dic[forest]=score1
    
                    print('params: %s score: %3f' %(forest, score1))

               if max_score < score1:
                    max_score = score1 


params: RandomForestClassifier(max_depth=1, n_estimators=1, random_state=1) score: 0.606061
params: RandomForestClassifier(max_depth=2, n_estimators=1, random_state=1) score: 0.606061
params: RandomForestClassifier(max_depth=3, n_estimators=1, random_state=1) score: 0.636364
params: RandomForestClassifier(max_depth=4, n_estimators=1, random_state=1) score: 0.636364
params: RandomForestClassifier(max_depth=5, n_estimators=1, random_state=1) score: 0.515152
params: RandomForestClassifier(max_depth=6, n_estimators=1, random_state=1) score: 0.560606
params: RandomForestClassifier(max_depth=7, n_estimators=1, random_state=1) score: 0.575758
params: RandomForestClassifier(max_depth=8, n_estimators=1, random_state=1) score: 0.530303
params: RandomForestClassifier(max_depth=9, n_estimators=1, random_state=1) score: 0.606061
params: RandomForestClassifier(max_depth=10, n_estimators=1, random_state=1) score: 0.484848
params: RandomForestClassifier(max_depth=11, n_estimators=1, random_state=1) sc

params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=2,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=2,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=16, n_estimators=2,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=17, n_estimators=2,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=18, n_estimators=2,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=2,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=2,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(max_depth=

params: RandomForestClassifier(criterion='entropy', max_depth=16, n_estimators=4,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=17, n_estimators=4,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=18, n_estimators=4,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=4,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=4,
                       random_state=1) score: 0.606061
params: RandomForestClassifier(max_depth=1, n_estimators=5, random_state=1) score: 0.621212
params: RandomForestClassifier(max_depth=2, n_estimators=5, random_state=1) score: 0.621212
params: RandomForestClassifier(max_depth=3, n_estimators=5, random_state=1) score: 0.621212
params: RandomForestClassifier(max_dept

                       random_state=1) score: 0.651515
params: RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=8,
                       random_state=1) score: 0.681818
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=8,
                       random_state=1) score: 0.681818
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=8,
                       random_state=1) score: 0.621212
params: RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=8,
                       random_state=1) score: 0.681818
params: RandomForestClassifier(criterion='entropy', max_depth=11, n_estimators=8,
                       random_state=1) score: 0.666667
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=8,
                       random_state=1) score: 0.651515
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=8,
                       random_state=1) score

params: RandomForestClassifier(criterion='entropy', max_depth=3, n_estimators=10,
                       random_state=1) score: 0.651515
params: RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=10,
                       random_state=1) score: 0.651515
params: RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=10,
                       random_state=1) score: 0.636364
params: RandomForestClassifier(criterion='entropy', max_depth=6, n_estimators=10,
                       random_state=1) score: 0.651515
params: RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=10,
                       random_state=1) score: 0.636364
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=10,
                       random_state=1) score: 0.727273
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=10,
                       random_state=1) score: 0.696970
params: RandomForestClassifier(criterion=

...

params: RandomForestClassifier(criterion='entropy', max_depth=17, n_estimators=42,
                       random_state=1) score: 0.787879
params: RandomForestClassifier(criterion='entropy', max_depth=18, n_estimators=42,
                       random_state=1) score: 0.772727
params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=42,
                       random_state=1) score: 0.772727
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=42,
                       random_state=1) score: 0.772727
params: RandomForestClassifier(max_depth=1, n_estimators=43, random_state=1) score: 0.575758
params: RandomForestClassifier(max_depth=2, n_estimators=43, random_state=1) score: 0.666667
params: RandomForestClassifier(max_depth=3, n_estimators=43, random_state=1) score: 0.651515
params: RandomForestClassifier(max_depth=4, n_estimators=43, random_state=1) score: 0.666667
params: RandomForestClassifier(max_depth=5, n_estimators=43, random_state=1)

params: RandomForestClassifier(criterion='entropy', max_depth=11, n_estimators=44,
                       random_state=1) score: 0.666667
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=44,
                       random_state=1) score: 0.772727
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=44,
                       random_state=1) score: 0.757576
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=44,
                       random_state=1) score: 0.757576
params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=44,
                       random_state=1) score: 0.787879
params: RandomForestClassifier(criterion='entropy', max_depth=16, n_estimators=44,
                       random_state=1) score: 0.772727
params: RandomForestClassifier(criterion='entropy', max_depth=17, n_estimators=44,
                       random_state=1) score: 0.772727
params: RandomForestClassifier(cri

params: RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=46,
                       random_state=1) score: 0.666667
params: RandomForestClassifier(criterion='entropy', max_depth=6, n_estimators=46,
                       random_state=1) score: 0.621212
params: RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=46,
                       random_state=1) score: 0.681818
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=46,
                       random_state=1) score: 0.742424
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=46,
                       random_state=1) score: 0.712121
params: RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=46,
                       random_state=1) score: 0.666667
params: RandomForestClassifier(criterion='entropy', max_depth=11, n_estimators=46,
                       random_state=1) score: 0.651515
params: RandomForestClassifier(criterio

params: RandomForestClassifier(max_depth=16, n_estimators=48, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=17, n_estimators=48, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=18, n_estimators=48, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=19, n_estimators=48, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=20, n_estimators=48, random_state=1) score: 0.681818
params: RandomForestClassifier(criterion='entropy', max_depth=1, n_estimators=48,
                       random_state=1) score: 0.575758
params: RandomForestClassifier(criterion='entropy', max_depth=2, n_estimators=48,
                       random_state=1) score: 0.651515
params: RandomForestClassifier(criterion='entropy', max_depth=3, n_estimators=48,
                       random_state=1) score: 0.666667
params: RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=48,
                       random_state=1

params: RandomForestClassifier(max_depth=6, n_estimators=50, random_state=1) score: 0.636364
params: RandomForestClassifier(max_depth=7, n_estimators=50, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=8, n_estimators=50, random_state=1) score: 0.666667
params: RandomForestClassifier(max_depth=9, n_estimators=50, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=10, n_estimators=50, random_state=1) score: 0.696970
params: RandomForestClassifier(max_depth=11, n_estimators=50, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=12, n_estimators=50, random_state=1) score: 0.696970
params: RandomForestClassifier(max_depth=13, n_estimators=50, random_state=1) score: 0.727273
params: RandomForestClassifier(max_depth=14, n_estimators=50, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=15, n_estimators=50, random_state=1) score: 0.681818
params: RandomForestClassifier(max_depth=16, n_estimators=50, ra

In [32]:
forest=RandomForestClassifier(criterion='entropy',  n_estimators=3, random_state= 1)

In [33]:
forest=RandomForestClassifier(criterion='entropy', max_depth=1, 
                       n_estimators=5, random_state=1)

In [34]:
forest.fit(X_train_std, y_train)

RandomForestClassifier(criterion='entropy', max_depth=1, n_estimators=5,
                       random_state=1)

In [35]:
forest.score(X_test_std, y_test)

0.6363636363636364

In [36]:
forest.score(X_train_std, y_train)

0.6381578947368421

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(estimator=forest,
                      X=X_train_std, y=y_train,
                      cv=5, n_jobs=1)

In [ ]:
score

In [ ]:
score.mean()

# Boosted forest

In [37]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [59]:
tree=DecisionTreeClassifier(criterion='entropy', max_depth=7, random_state=1)
ada=AdaBoostClassifier(base_estimator=tree, n_estimators=96, learning_rate=0.1, random_state=1)

In [60]:
ada.fit(X_train_std, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=7,
                                                         random_state=1),
                   learning_rate=0.1, n_estimators=96, random_state=1)

In [61]:
ada.score(X_train_std, y_train)

1.0

In [62]:
ada.score(X_test_std, y_test)

0.5

In [58]:
from sklearn.datasets import load_digits
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

def main():
    digits = load_digits()
    ada = AdaBoostClassifier(n_estimators=100) # ついでに増やす
    params = {"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
             
    }
    cv = GridSearchCV(ada, params, cv=10, n_jobs=-1, verbose=2)

    cv.fit(X_train_std, y_train)
    
    print(cv.best_params_)
    pred = cv.predict(X_test_std)
    print(classification_report(y_test, pred))

if __name__ == "__main__":
    main()

Fitting 10 folds for each of 1485 candidates, totalling 14850 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done 773 tasks      | elapsed:   40.3s
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1813 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2514 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  3.5min


KeyboardInterrupt: 

# xgboost

In [ ]:
import xgboost as xgb

In [391]:
dtrain = xgb.DMatrix(X_train_std, label=y_train)
param = {'max_depth': 100, 'eta': 0.5, 'objective': 'multi:softmax', 'num_class': 2} 
num_round = 100 
bst = xgb.train(param, dtrain, num_round)  

In [392]:
dtest = xgb.DMatrix(X_test_std)  
pred = bst.predict(dtest) 
score = accuracy_score(y_test, pred)
print('score:{0:.4f}'.format(score))  

score:0.6061
